In [1]:
import model
from Augment import train_datagen
import joblib
import shelve
import tensorflow as tf
import time
from math import ceil
from config import vocabulary
import helper
import matplotlib.pyplot as plt
import numpy as np

In [2]:
batch_size = 32

In [3]:
mount_point = '../'

In [4]:
with shelve.open(mount_point+'IAM_Data','c') as shelf:
    train_label = shelf['train_label']

In [5]:
train_array = joblib.load(mount_point+'data/train_array')

In [6]:
n_epochs = 10

In [7]:
graph,dropout_conv,dropout_lstm,dropout_fc,inputs,time_steps,targets,loss,train,decoded,label_error_rate,seq_len = model.model()

Convolution_Output_size:(5,29)
LSTM_Output_size:(Tensor("bidirectional_rnn/fw/fw/transpose_1:0", shape=(?, 145, 256), dtype=float32),Tensor("ReverseV2:0", shape=(?, 145, 256), dtype=float32))


In [8]:
!rm -rf ../Augmented/*

In [9]:
train_generator = train_datagen.flow(train_array,train_label,batch_size,save_to_dir=mount_point+'Augmented',save_prefix='transformed')

In [10]:
num_training_samples = train_array.shape[0]

In [11]:
num_batches = int(ceil(num_training_samples/batch_size))

In [12]:
with tf.Session(graph = graph) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = False
    timer  = 0
    
    #Epoch Loop
    for e in range(n_epochs):
        start_time = time.time()
        
        if (e%1)==0:
            checkpoint = True
        
        total_cost = 0.0
        total_ler = 0.0    
    
        count = 0
        
        #Mini Batch loop
        for x,y in train_generator:
            
            actual_batch_size = x.shape[0]
            
            if count == num_batches:
                count = 0
                break

            sparse_y = helper._batch_y(y,vocabulary)

            feed_train = {
                         inputs:x,targets:sparse_y,
                         time_steps:np.array([seq_len]*actual_batch_size),
                         dropout_conv:0.5,dropout_fc:0.5,dropout_lstm:0.5,
                    }
            
            _,cost_val,train_ler_val = sess.run([train,loss,label_error_rate],feed_dict= feed_train)

            print(cost_val,train_ler_val)
            
            total_cost += cost_val
            total_ler += train_ler_val

            count+=1

        if checkpoint:
            end_time = time.time()
            time_taken = end_time - start_time
            timer += time_taken

            print(total_cost,total_ler)        

509.658 2.90589
333.427 1.38683
533.768 1.23833
891.671 2.71974
693.905 2.39574
621.473 2.4874
1268.61 1.29103
705.205 1.98308
511.479 1.33557
384.837 1.78678
338.656 1.23752
265.909 1.33524
251.724 1.38148
252.915 1.26739
243.611 1.1522
249.366 1.09277
240.243 1.27858
239.398 1.13717
242.522 1.08651
240.759 1.16391
234.789 1.1104
230.613 1.17361
230.214 1.10801
232.723 1.20501
233.729 1.13669
229.151 1.29565
227.055 1.13654
224.706 1.10483
226.692 1.10779
227.629 1.18431
231.47 1.20448
219.021 1.15057
222.265 1.10997
226.382 1.21455
221.836 1.09189
226.32 1.2081
220.237 1.07289
223.806 1.16489
220.61 1.10808
229.45 1.166
222.784 1.08583
221.279 1.06979
220.752 1.07668
221.955 1.15492
226.551 1.05742
239.35 1.23483
217.155 1.1521
229.48 1.0452
221.118 1.16279
221.157 1.06415
215.272 1.08079


KeyboardInterrupt: 

In [ ]:
%matplotlib inline

In [ ]:
# img = train_array[0].reshape(train_array[0].shape[:2])

In [ ]:
# plt.imshow(img,cmap='gray')

In [ ]:
# train_label[0]